In [1]:
import numpy as np
import pickle
import random
import copy

In [2]:
# read the raw data and transform it to an image format
with open('test_batch', 'rb') as fo:
    dict = pickle.load(fo, encoding='latin1')

test_images = dict["data"].reshape((-1, 3, 32, 32)).astype("float32")
test_images = test_images.transpose(0,2,3,1).astype(int)

In [3]:
# generate attack images

np.random.seed(3787)
select_attacks = np.random.choice(len(test_images), size=20, replace=False)
rem_indices = np.setdiff1d(np.arange(len(test_images)), select_attacks)
test_images = test_images[rem_indices]

attack_repeats = np.random.randint(5, 12, size=len(select_attacks))

attack_images = []
mean = 0
sigma = 5
for ii, idx in enumerate(select_attacks):
    for jj in range(attack_repeats[ii]):
      gauss = np.round(np.random.normal(mean,sigma,test_images[0].shape)).astype(int)
      attack_images.append(np.clip(test_images[idx] + gauss, a_min=0, a_max=255))

attack_images = np.asarray(attack_images)
attack_images_indices = np.arange(len(attack_images))
np.random.shuffle(attack_images_indices)
attack_images = attack_images[attack_images_indices]

test_images_indices = np.arange(len(test_images))
np.random.shuffle(test_images_indices)
test_images = test_images[test_images_indices]

test_images_w_attack = np.vstack((test_images, attack_images)).astype(np.uint8)

print(attack_images.shape)

(166, 32, 32, 3)


In [4]:
# distribute the images between good and bad users

random.seed(8112)
np.random.seed(398)

num_good_users = 500
num_bad_users = 20

def split_indices(chunk_sizes, start_idx=0):

    cur_idx = start_idx
    chunk_indices = []
    for ii, chunk_size in enumerate(chunk_sizes):
        chunk_indices.append(np.arange(cur_idx, cur_idx+chunk_sizes[ii]))
        cur_idx += chunk_size
    return chunk_indices

all_user_ids = np.arange(num_good_users+num_bad_users)
good_users_ids = np.random.choice(all_user_ids, size=num_good_users, replace=False)
bad_users_ids = np.setdiff1d(all_user_ids, good_users_ids)

good_users_data = {ii:[] for ii in good_users_ids}
bad_users_data = {ii:[] for ii in bad_users_ids}

all_good_indices = np.arange(len(test_images))
all_bad_indices = np.arange(len(attack_images))

good_chunk_sizes = np.random.randint(low=9, high=30, size=(num_good_users))
bad_user_good_chunk_sizes = np.random.randint(low=4, high=18, size=(num_bad_users))
bad_chunk_sizes = attack_repeats

good_chunk_indices = split_indices(good_chunk_sizes)
bad_user_good_chunk_indices = split_indices(bad_user_good_chunk_sizes, start_idx=sum(good_chunk_sizes))

bad_chunk_indices = split_indices(bad_chunk_sizes, start_idx=len(test_images))

for ii, un in enumerate(good_users_data):
    good_users_data[un].extend(good_chunk_indices[ii])

for ii, un in enumerate(bad_users_data):
    bad_users_data[un].extend(bad_user_good_chunk_indices[ii])
    bad_users_data[un].extend(bad_chunk_indices[ii])

In [5]:
# shuffle the image indices

all_indices = np.arange(len(test_images_w_attack))
np.random.shuffle(all_indices)
index_mapping = {ii:np.where(all_indices == ii)[0][0] for ii in np.arange(len(test_images_w_attack))}

test_images_w_attack_shuffled = test_images_w_attack[all_indices]

good_users_data_remapped = copy.deepcopy(good_users_data)

for un in good_users_data:
    new_indices = np.asarray([index_mapping[ii] for ii in good_users_data[un]])
    np.random.shuffle(new_indices)
    good_users_data_remapped[un] = new_indices

bad_users_data_remapped = copy.deepcopy(bad_users_data)

for un in bad_users_data:
    try:
        new_indices = np.asarray([index_mapping[ii] for ii in bad_users_data[un]])
    except:
        print(un)
    np.random.shuffle(new_indices)
    bad_users_data_remapped[un] = new_indices

In [6]:
# generate challlenge data format


with open('model_queries.npy', 'wb') as fp:
    np.save(fp, test_images_w_attack_shuffled)

# python dicts preserve insert order, shuffle the keys
user_query_indices = {**good_users_data_remapped, **bad_users_data_remapped}
user_names = sorted(list(user_query_indices.keys()))

user_queryies_str = '\n'.join([f"{','.join([str(i) for i in user_query_indices[un]])}" for un in user_names])

fp = open('user_query_indices.txt', 'w')
fp.write(user_queryies_str)
fp.close()

In [7]:
bad_users_string = f','.join([str(i) for i in np.sort(bad_users_ids)])
flag = f'ictf{{{bad_users_string}}}'

fp = open('flag.txt', 'w')
fp.write(flag)
fp.close()